<a href="https://colab.research.google.com/github/marcusamartin/CS205_8Puzzle/blob/main/CS205_8Puzzle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def main():
    print("Welcome to my 8-Puzzle Solver.\n")
    puzzleChoice = int(input("Type '1' to use a default puzzle, or '2' to create your own.\n"))
    puzzle = [[], [], []]
    if (puzzleChoice == 1):
        puzzle = [['1', '2', '3'], ['4', '5', '6'], ['7', '8', '9']]
    elif (puzzleChoice == 2):
        print("Enter your puzzle, using a zero to represent the blank. Please only enter valid 8-puzzles. Enter the puzzle delimiting the numbers with a space. RET only when finished.\n")
        firstRow = input("Enter the first row: ").split()
        secondRow = input("Enter the second row: ").split()
        thirdRow = input("Enter the third row: ").split()
        puzzle = [firstRow, secondRow, thirdRow]
        
    algorithm = int(input("Select algorithm. (1) for Uniform Cost Search, (2) for the Misplaced Tile Heuristic, or (3) the Manhattan Distance Heuristic.\n"))
    
    print("\n")
    print(puzzle)
    print("\n")
    print(algorithm)

In [ ]:
main()

Welcome to my 8-Puzzle Solver.

Type '1' to use a default puzzle, or '2' to create your own.
1
Select algorithm. (1) for Uniform Cost Search, (2) for the Misplaced Tile Heuristic, or (3) the Manhattan Distance Heuristic.
3


[['1', '2', '3'], ['4', '5', '6'], ['7', '8', '9']]


3
